In [8]:
!nvidia-smi --help

NVIDIA System Management Interface -- v511.65

NVSMI provides monitoring information for Tesla and select Quadro devices.
The data is presented in either a plain text or an XML format, via stdout or a file.
NVSMI also provides several management operations for changing the device state.

Note that the functionality of NVSMI is exposed through the NVML C-based
library. See the NVIDIA developer website for more information about NVML.
Python wrappers to NVML are also available.  The output of NVSMI is
not guaranteed to be backwards compatible; NVML and the bindings are backwards
compatible.

http://developer.nvidia.com/nvidia-management-library-nvml/
http://pypi.python.org/pypi/nvidia-ml-py/
Supported products:
- Full Support
    - All Tesla products, starting with the Kepler architecture
    - All Quadro products, starting with the Kepler architecture
    - All GRID products, starting with the Kepler architecture
    - GeForce Titan products, starting with the Kepler architecture
- Limi

In [7]:
gpu_info = !nvidia-smi --list-gpus
print(gpu_info)

['GPU 0: NVIDIA GeForce RTX 3080 Laptop GPU (UUID: GPU-5a91edd9-4caa-7276-1907-0d78ca6eb8f3)']


In [65]:
gpu_detail = !nvidia-smi --query --id=0 --xml-format
detailmatcher = re.compile('^<([a-z_]+)>(.+)<\/[a-z_]+>')

for line in gpu_detail:
    cleanline = line.strip()
    keyvalue = detailmatcher.findall(cleanline)
    
    if len(keyvalue) == 0:
        continue

    if keyvalue[0][0] == "driver_version":
        print(f'  Driver Version: {keyvalue[0][1]}')
    if keyvalue[0][0] == "cuda_version":
        print(f'  CUDA Version: {keyvalue[0][1]}')
    if keyvalue[0][0] == "product_brand":
        print(f'  Brand: {keyvalue[0][1]}')
    if keyvalue[0][0] == "product_architecture":
        print(f'  Architecture: {keyvalue[0][1]}')
    if keyvalue[0][0] == "total":
        print(f'  Memory: {keyvalue[0][1]}')

  Driver Version: 511.65
  CUDA Version: 11.6
  Brand: GeForce
  Architecture: Ampere
  Memory: 8192 MiB
  Memory: 16384 MiB


In [64]:
import re

regex = r"^GPU ([0-9+]): (.+) \(UUID: ([a-zA-Z0-9-]+)\)$"
matcher = re.compile(regex)
gpus_infos = !nvidia-smi --list-gpus

for gpu in gpus_info:
    gpu_info = matcher.findall(gpu)
    gpu_id = gpu_info[0][0]
    gpu_name = gpu_info[0][1]
    print(f'GPU {gpu_id}: {gpu_name}')
    
    gpu_detail = !nvidia-smi --query --id=$gpu_id --xml-format
    detailmatcher = re.compile('^<([a-z_]+)>(.+)<\/[a-z_]+>')

    for line in gpu_detail:
        cleanline = line.strip()
        keyvalue = detailmatcher.findall(cleanline)
        
        if len(keyvalue) == 0:
            continue

        if keyvalue[0][0] == "driver_version":
            print(f'  Driver Version: {keyvalue[0][1]}')
        if keyvalue[0][0] == "cuda_version":
            print(f'  CUDA Version: {keyvalue[0][1]}')
        if keyvalue[0][0] == "product_brand":
            print(f'  Brand: {keyvalue[0][1]}')
        if keyvalue[0][0] == "product_architecture":
            print(f'  Architecture: {keyvalue[0][1]}')
        if keyvalue[0][0] == "total":
            print(f'  Memory: {keyvalue[0][1]}')
    

GPU 0: NVIDIA GeForce RTX 3080 Laptop GPU
  Driver Version: 511.65
  CUDA Version: 11.6
  Brand: GeForce
  Architecture: Ampere
  Memory: 8192 MiB
  Memory: 16384 MiB


In [7]:
!nvidia-smi --query --xml-format

<?xml version="1.0" ?>
<!DOCTYPE nvidia_smi_log SYSTEM "nvsmi_device_v11.dtd">
<nvidia_smi_log>
	<timestamp>Sat Feb 12 21:45:36 2022</timestamp>
	<driver_version>511.65</driver_version>
	<cuda_version>11.6</cuda_version>
	<attached_gpus>1</attached_gpus>
	<gpu id="00000000:01:00.0">
		<product_name>NVIDIA GeForce RTX 3080 Laptop GPU</product_name>
		<product_brand>GeForce</product_brand>
		<product_architecture>Ampere</product_architecture>
		<display_mode>Disabled</display_mode>
		<display_active>Disabled</display_active>
		<persistence_mode>N/A</persistence_mode>
		<mig_mode>
			<current_mig>N/A</current_mig>
			<pending_mig>N/A</pending_mig>
		</mig_mode>
		<mig_devices>
			None
		</mig_devices>
		<accounting_mode>Disabled</accounting_mode>
		<accounting_mode_buffer_size>4000</accounting_mode_buffer_size>
		<driver_model>
			<current_dm>WDDM</current_dm>
			<pending_dm>WDDM</pending_dm>
		</driver_model>
		<serial>N/A</serial>
		<uuid>GPU-5a91edd9-4caa-7276-1907-0d78ca6eb8f3</uuid>


In [29]:
import xml.etree.ElementTree as ET

gpu_detail = !nvidia-smi --query --xml-format

root = ET.ElementTree(ET.fromstringlist(gpu_detail)).getroot()
for child in root:
    if child.tag == 'driver_version':
        print(f'NVIDIA Driver Version: {child.text}')
    elif child.tag == 'cuda_version':
        print(f'CUDA Version: {child.text}')
    elif child.tag == 'gpu':
        print(f'{child.attrib["id"]}:')
        for gpu_property in child:
            if gpu_property.tag == 'product_name':
                print(f'  Product Name : {gpu_property.text}')
            elif gpu_property.tag == 'product_brand':
                print(f'  Brand        : {gpu_property.text}')
            elif gpu_property.tag == 'product_architecture':
                print(f'  Architecture : {gpu_property.text}')
            elif gpu_property.tag == 'uuid':
                print(f'  UUID         : {gpu_property.text}')
            elif gpu_property.tag == 'fb_memory_usage':
                for mem in gpu_property:
                    if mem.tag == 'total':
                        print(f'  Total Memory : {mem.text}')

NVIDIA Driver Version: 511.65
CUDA Version: 11.6
00000000:01:00.0:
  Product Name : NVIDIA GeForce RTX 3080 Laptop GPU
  Brand        : GeForce
  Architecture : Ampere
  UUID         : GPU-5a91edd9-4caa-7276-1907-0d78ca6eb8f3
  Total Memory : 8192 MiB
